In [ ]:
import PySpin

In [ ]:
system = PySpin.System.GetInstance()
cam_list = system.GetCameras()
print('# cameras detected: '+str(cam_list.GetSize()))
cam = cam_list[0]
cam.Init()

In [ ]:
cam.DeInit()
system.ReleaseInstance()

In [ ]:
assert cam.ExposureAuto.GetAccessMode() == PySpin.RW

cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
    print('Automatic exposure disabled...')

exposure_us=2*1e6
cam.ExposureTime.SetValue(exposure_us)

assert cam.AcquisitionMode.GetAccessMode() == PySpin.RW



In [ ]:
cam.AcquisitionMode.SetValue(PySpin.AcquisitionMode_SingleFrame)
timestr=time.strftime('%Y-%m-%d-%H-%M-%S')


for exp_ms in [0.5, 1.0, 2.0, 4.0]:
    cam.ExposureTime.SetValue(exp_ms*1e6)
    cam.BeginAcquisition()
    image = cam.GetNextImage()
    filename = timestr+'_%1.2fms.png' % (exp_ms)
    image.Save(filename)
    print('Saved '+filename)
    image.Release()
    cam.EndAcquisition()


In [ ]:
def configure_exposure(cam,exp=):
    """
     This function configures a custom exposure time. Automatic exposure is turned
     off in order to allow for the customization, and then the custom setting is
     applied.

     :param cam: Camera to configure exposure for.
     :type cam: CameraPtr
     :return: True if successful, False otherwise.
     :rtype: bool
    """

    print('*** CONFIGURING EXPOSURE ***\n')

    try:
        result = True

        # Turn off automatic exposure mode
        #
        # *** NOTES ***
        # Automatic exposure prevents the manual configuration of exposure
        # times and needs to be turned off for this example. Enumerations
        # representing entry nodes have been added to QuickSpin. This allows
        # for the much easier setting of enumeration nodes to new values.
        #
        # The naming convention of QuickSpin enums is the name of the
        # enumeration node followed by an underscore and the symbolic of
        # the entry node. Selecting "Off" on the "ExposureAuto" node is
        # thus named "ExposureAuto_Off".
        #
        # *** LATER ***
        # Exposure time can be set automatically or manually as needed. This
        # example turns automatic exposure off to set it manually and back
        # on to return the camera to its default state.

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic exposure. Aborting...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
        print('Automatic exposure disabled...')

        # Set exposure time manually; exposure time recorded in microseconds
        #
        # *** NOTES ***
        # Notice that the node is checked for availability and writability
        # prior to the setting of the node. In QuickSpin, availability and
        # writability are ensured by checking the access mode.
        #
        # Further, it is ensured that the desired exposure time does not exceed
        # the maximum. Exposure time is counted in microseconds - this can be
        # found out either by retrieving the unit with the GetUnit() method or
        # by checking SpinView.

        if cam.ExposureTime.GetAccessMode() != PySpin.RW:
            print('Unable to set exposure time. Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        exposure_time_to_set = 2000000.0
        exposure_time_to_set = min(cam.ExposureTime.GetMax(), exposure_time_to_set)
        cam.ExposureTime.SetValue(exposure_time_to_set)
        print('Shutter time set to %s us...\n' % exposure_time_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [ ]:
def acquire_images(cam):
    """
    This function acquires and saves 10 images from a device; please see
    Acquisition example for more in-depth comments on the acquisition of images.

    :param cam: Camera to acquire images from.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    print('*** IMAGE ACQUISITION ***')

    try:
        result = True

        # Set acquisition mode to continuous
        if cam.AcquisitionMode.GetAccessMode() != PySpin.RW:
            print('Unable to set acquisition mode to continuous. Aborting...')
            return False

        cam.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
        print('Acquisition mode set to continuous...')

        # Begin acquiring images
        cam.BeginAcquisition()

        print('Acquiring images...')

        # Get device serial number for filename
        device_serial_number = ''
        if cam.TLDevice.DeviceSerialNumber is not None and cam.TLDevice.DeviceSerialNumber.GetAccessMode() == PySpin.RO:
            device_serial_number = cam.TLDevice.DeviceSerialNumber.GetValue()

            print('Device serial number retrieved as %s...' % device_serial_number)

        #Retrieve, convert, and save images
        for i in range(3):

            try:
                # Retrieve next received image and ensure image completion
                image_result = cam.GetNextImage()

                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d...' % image_result.GetImageStatus())

                else:
                    # Print image information
                    width = image_result.GetWidth()
                    height = image_result.GetHeight()
                    print('Grabbed Image %d, width = %d, height = %d' % (i, width, height))

                    # Convert image to Mono8
                    #image_converted = image_result.Convert(PySpin.PixelFormat_Mono8)
                    image_converted = image_result.Convert(PySpin.PixelFormat_RGB32f)                    

                    # Create a unique filename
                    filename = 'larvaecam-%d.tif' % (i)

                    # Save image
                    image_converted.Save(filename)

                    print('Image saved at %s' % filename)

                # Release image
                image_result.Release()

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                result = False

        # End acquisition
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [ ]:
def acquire_image(cam):


    try:
        result = True


        print('Acquiring images...')

        try:
            # Retrieve next received image and ensure image completion
            image_result = cam.GetNextImage()

            if image_result.IsIncomplete():
                print('Image incomplete with image status %d...' % image_result.GetImageStatus())
                
            return image_result
 
        except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                result = False

        # End acquisition
        #cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [ ]:
configure_exposure(cam)


In [ ]:
i=acquire_image(cam)

In [ ]:
cam.BeginAcquisition()

In [ ]:
image_result = cam.GetNextImage()

In [ ]:
image_result.GetPixelFormat()

In [ ]:
PySpin.PixelFormat_BayerRG8

In [ ]:
image_result.Save('img2.png')
image_result.Release()

In [ ]:
cam.EndAcquisition()

In [ ]:
import time

In [ ]:
time.strftime('%Y-%m-%d-%H-%M-%S')

In [ ]:
1*1e6

In [ ]:
cam.